## Metodo con Matrices triangulares A = L.U

El objetivo de este notebook es resolver un problema de sistema de ecuaciones lineales utilizando el metodo de factorización A = LU, donde L es la matriz tringular inferior y U es la matriz triangular superior


In [400]:
import numpy as np
import random

In [401]:
n = random.randint(2, 4)
A = np.random.randint(1, 10, size=(n, n))

# Generar un vector aleatorio de longitud n
B = np.random.randint(1, 10, size=n)

print("Matriz:")
print(A)
print("Vector:")
print(B)

Matriz:
[[7 2 1 1]
 [9 9 8 3]
 [7 4 8 9]
 [5 6 2 3]]
Vector:
[7 6 1 6]


In [402]:
# #A = np.array([[ 3. , -0.1, -0.2],
#               [ 0.1,  7. , -0.3],
#               [ 0.3, -0.2, 10. ]], dtype=float)

# #B = np.array([7.85,-19.3,71.4], dtype=float)

In [403]:
B  = np.transpose([B]) # Se transpone el vector para poder hacer la concatenación con la matriz
AB = np.concatenate((A,B),axis=1) # Se concatena la matriz con el vector
AB = np.copy(AB)  # Se genera una copia de la matriz aumentada

In [404]:
# Pivoteo parcial por filas
tamano = np.shape(AB)
n = tamano[0]
m = tamano[1]

Luego debemos de intercambiar las filas en la matriz extendida AB para asegurarse de que el elemento en la posición, i,i (el pivote) sea el mayor en valor absoluto en su columna. Esto ayuda a evitar divisiones por cero y mejora la estabilidad numérica del algoritmo de eliminación hacia adelante.

In [405]:
# Para cada fila en AB
for i in range(0,n-1,1): # Para cada fila en AB, se comienza desde la primer fila hasta la penultima fila

    # columna desde diagonal i en adelante
    columna = abs(AB[i:,i]) # Se optine la columna correspondiente al pivote en valor absoluto
    dondemax = np.argmax(columna)# Se encuentra la posición del elemento máximo en la columna

    # Si la posición del elemento máximo no es 0(es decir, el elemto máximo no esta en la diagonal), se intercambian las filas i y dondemax + i
    # dondemax no está en diagonal
    if (dondemax !=0):
        # intercambia filas
        temporal = np.copy(AB[i,:])
        AB[i,:] = AB[dondemax+i,:]
        AB[dondemax+i,:] = temporal

AB1 = np.copy(AB)
A1 = np.copy(AB[:,:m-1])
B1 = np.copy(AB[:,m-1])

In [406]:
AB1  # Matriz reemplazada


array([[9, 9, 8, 3, 6],
       [5, 6, 2, 3, 6],
       [7, 4, 8, 9, 1],
       [7, 2, 1, 1, 7]])

In [407]:
A1 # Matriz reemplazada ni el vector inicial 

array([[9, 9, 8, 3],
       [5, 6, 2, 3],
       [7, 4, 8, 9],
       [7, 2, 1, 1]])

In [408]:
B1 ## Vector resultado inicial 

array([6, 6, 1, 7])

Realizamos la eliminación hacia adelante en la matriz extendida AB para convertirla en una matriz triangular superior. Al mismo tiempo, se construye una matriz L que contiene los factores de multiplicación utilizados en cada paso de la eliminación hacia adelante. La matriz L es una matriz triangular inferior con unos en la diagonal principal.

In [409]:
# eliminacion hacia adelante
# se inicializa L
L = np.identity(n,dtype=float)
for i in range(0,n-1,1):
    pivote = AB[i,i]
    adelante = i+1
    for k in range(adelante,n,1):
        factor = AB[k,i]/pivote
        AB[k,:] = AB[k,:] - AB[i,:]*factor
        L[k,i] = factor

U = np.copy(AB[:,:m-1])

In [410]:
U # Triangular superior

array([[  9,   9,   8,   3],
       [  0,   1,  -2,   1],
       [  0,   0,  -5,   9],
       [  0,   0,   0, -23]])

In [411]:
L # Triangular inferior

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.55555556,  1.        ,  0.        ,  0.        ],
       [ 0.77777778, -3.        ,  1.        ,  0.        ],
       [ 0.77777778, -5.        ,  3.        ,  1.        ]])

In [412]:
# Resolver LY = B   donde Y = B1
B2  = np.transpose([B1])
AB =np.concatenate((L,B2),axis=1)

In [413]:
B2

array([[6],
       [6],
       [1],
       [7]])

In [414]:
AB   # AB = LY 

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  6.        ],
       [ 0.55555556,  1.        ,  0.        ,  0.        ,  6.        ],
       [ 0.77777778, -3.        ,  1.        ,  0.        ,  1.        ],
       [ 0.77777778, -5.        ,  3.        ,  1.        ,  7.        ]])

In [415]:
# Vamos a resolver LY
# sustitución hacia adelante
Y = np.zeros(n,dtype=float)
Y[0] = AB[0,n]
for i in range(1,n,1):
    suma = 0
    for j in range(0,i,1):
        suma = suma + AB[i,j]*Y[j]
    b = AB[i,n]
    Y[i] = (b-suma)/AB[i,i]

Y = np.transpose([Y])
Y

array([[6.        ],
       [2.66666667],
       [4.33333333],
       [2.66666667]])

In [416]:
# Resolver UX = Y   donde Y=B1
AB =np.concatenate((U,Y),axis=1)
AB

array([[  9.        ,   9.        ,   8.        ,   3.        ,
          6.        ],
       [  0.        ,   1.        ,  -2.        ,   1.        ,
          2.66666667],
       [  0.        ,   0.        ,  -5.        ,   9.        ,
          4.33333333],
       [  0.        ,   0.        ,   0.        , -23.        ,
          2.66666667]])

In [417]:
# sustitución hacia atrás
ultfila = n-1
ultcolumna = m-1
X = np.zeros(n,dtype=float)

for i in range(ultfila,0-1,-1):
    suma = 0
    for j in range(i+1,ultcolumna,1):
        suma = suma + AB[i,j]*X[j]
    b = AB[i,ultcolumna]
    X[i] = (b-suma)/AB[i,i]

X = np.transpose([X])
X

array([[ 1.02930757],
       [ 0.63188406],
       [-1.07536232],
       [-0.11594203]])

In [418]:
# SALIDA
print('Pivoteo parcial por filas: AB')
print(AB1)
print('eliminación hacia adelante')
print('Matriz U: ')
print(U)
print('matriz L: ')
print(L)
print('B1 :')
print(B1)
print("Y Sustitución hacia adelante - Solución")
print(Y)
print('X Sustitución hacia atras - Solución')
print(X)

Pivoteo parcial por filas: AB
[[9 9 8 3 6]
 [5 6 2 3 6]
 [7 4 8 9 1]
 [7 2 1 1 7]]
eliminación hacia adelante
Matriz U: 
[[  9   9   8   3]
 [  0   1  -2   1]
 [  0   0  -5   9]
 [  0   0   0 -23]]
matriz L: 
[[ 1.          0.          0.          0.        ]
 [ 0.55555556  1.          0.          0.        ]
 [ 0.77777778 -3.          1.          0.        ]
 [ 0.77777778 -5.          3.          1.        ]]
B1 :
[6 6 1 7]
Y Sustitución hacia adelante - Solución
[[6.        ]
 [2.66666667]
 [4.33333333]
 [2.66666667]]
X Sustitución hacia atras - Solución
[[ 1.02930757]
 [ 0.63188406]
 [-1.07536232]
 [-0.11594203]]


Ahora vamos a solucionar la Matriz L (triangular inferior), donde, en caso de infinitas soluciones, retorna una solución partular y una base para el espacio nulo de L.

In [419]:
#se utiliza para encontrar una solución única para una matriz triangular inferior L y un vector B1 dados
def unica_solucion(L, B1):
    n = len(L)
    x = np.zeros(n)

    for i in range(n):
        x[i] = B1[i]
        for j in range(i):
            x[i] -= L[i][j] * x[j]
        x[i] /= L[i][i]

    return x

#se utiliza para encontrar un número infinito de soluciones para una matriz triangular inferior L y un vector B1 dados.
def infinitas_soluciones(L, B1):
    n = len(L)
    x = np.zeros(n)

    free_vectors_bases = {}
    has_free_vectors = False
    for i in range(n):
        if L[i][i] == 0:
            net = B1[i]
            for j in range(i):
                net -= L[i][j] * x[j]
            if net != 0:
                raise Exception("No existe solución")
            free_vector_base = np.zeros(n)
            free_vector_base[i] = 1
            free_vectors_bases[i] = free_vector_base
            has_free_vectors = True
        else:
            if has_free_vectors:
                x[i] += B1[i] / L[i][i]
                for j in range(i):
                    if j in free_vectors_bases:
                        free_vectors_bases[j][i] -= (L[i][j]) / L[i][i]
                    else:
                        x[i] -= (L[i][j] * x[j]) / L[i][i]
            else:
                x[i] = B1[i]
                for j in range(i):
                    x[i] -= L[i][j] * x[j]
                x[i] /= L[i][i]

    return x, list(free_vectors_bases.values())

# Se hace el metodo de sustitución hacia adelante
def sustitucion_adelante(L, B1):
    diag = np.diag(L)
    has_infinite_solutions = np.any(diag == 0)
    if has_infinite_solutions:
        x, free_vectors_bases = infinitas_soluciones(L, B1)
        print("Si existen soluciones infinitas")
        print("La solución particular es:", x)
        print("La base para el espacio nulo:", free_vectors_bases)
        return x, free_vectors_bases
    else:
        result = unica_solucion(L, B1)
        print("Existe solución única:\n", result)
        return result

In [420]:
result = sustitucion_adelante(L, B1)

Existe solución única:
 [6.         2.66666667 4.33333333 2.66666667]
